---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

## Reading data

In [2]:
def read_data(file):
    input_df = pd.read_csv(file, encoding='mac_roman')    
    label = None
    if(file == 'train.csv'):
        input_df.dropna(how='all', inplace=True)
        label = input_df['compliance'].apply(lambda x: 1 if x == 1.0 else 0)
    feature_list = ['ticket_id','agency_name','inspector_name','violator_name',
                    'violation_street_number', 'violation_street_name', 'violation_zip_code',
                    'mailing_address_str_number','mailing_address_str_name', 'city', 'state', 'zip_code', 'country',
                    'ticket_issued_date', 'hearing_date',
                    'violation_code','disposition','fine_amount', 'admin_fee', 'state_fee', 'late_fee', 
                    'discount_amount', 'judgment_amount', 
                    'grafitti_status', 'clean_up_cost'
                   ]
    data_df = input_df[feature_list]
    '''
    latlon_df = pd.read_csv('latlons.csv', encoding='mac_roman')
    address_df = pd.read_csv('addresses.csv', encoding='mac_roman')
    geo_df = pd.merge(address_df, latlon_df, how='left', left_on='address', right_on='address')
    data_df = pd.merge(data_df, geo_df, how='left', left_on='ticket_id', right_on='ticket_id')
    data_df = data_df.drop('address', axis=1)
    '''
    data_df = data_df.set_index('ticket_id')
    data_df['grafitti_status'] = data_df['grafitti_status'].map(lambda x: 0 if pd.isnull(x) else 1)
    data_df['fine_amount'] = data_df['fine_amount'].map(lambda x: 0 if pd.isnull(x) else x) 
    
    return data_df, label
read_data('train.csv')[0]

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,violation_code,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,grafitti_status,clean_up_cost
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,...,9-1-36(a),Responsible by Default,250.0,20.0,10.0,25.0,0.0,305.0,0,0.0
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,...,61-63.0600,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,855.0,0,0.0
22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,MI,...,9-1-36(a),Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0,0.0
22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,MI,...,9-1-36(a),Not responsible by City Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0,0.0
22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,MI,...,9-1-36(a),Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0,0.0
22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,CA,...,9-1-36(a),Responsible by Default,250.0,20.0,10.0,25.0,0.0,305.0,0,0.0
18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,MI,...,61-63.0500,Responsible by Default,750.0,20.0,10.0,75.0,0.0,855.0,0,0.0
18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,MI,...,61-63.0100,Responsible by Default,100.0,20.0,10.0,10.0,0.0,140.0,0,0.0
18733,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,MI,...,61-63.0100,Responsible by Default,100.0,20.0,10.0,10.0,0.0,140.0,0,0.0


## Encode data

In [3]:
d = None
def encode_blight(df):
    from collections import defaultdict
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import MinMaxScaler
    d = defaultdict(LabelEncoder)
    scaler = MinMaxScaler()

    # Encoding the variable

    fit = df.apply(lambda x: d[x.name].fit_transform(x.factorize()[0]))

    # Inverse the encoded

    fit.apply(lambda x: d[x.name].inverse_transform(x.factorize()[0]))

    # Using the dictionary to label future data

    df_fit = df.apply(lambda x: d[x.name].transform(x.factorize()[0])).copy()
    
    df_fit = df_fit.apply(lambda x: scaler.fit_transform(x))
    return df_fit

In [29]:
def blight_model():    
    X_train, y_train = read_data('train.csv')
    X_test, y_test = read_data('test.csv')
    
    X_train = encode_blight(X_train)
    X_test = encode_blight(X_test)
    
    from sklearn.neural_network import MLPClassifier

    gs = MLPClassifier(activation = 'relu', hidden_layer_sizes = [32, 32], alpha=0.01, verbose=True)
    gs.fit(X_train, y_train)
    
    predict_prob = gs.predict_proba(X_test)
    print(gs.classes_)
    X_test['compliance'] = predict_prob[:,1].tolist()
    return X_test['compliance']
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape y

Iteration 1, loss = 0.16064083
Iteration 2, loss = 0.14057486
Iteration 3, loss = 0.13565658
Iteration 4, loss = 0.13287957
Iteration 5, loss = 0.13198308
Iteration 6, loss = 0.13077261
Iteration 7, loss = 0.13008641
Iteration 8, loss = 0.12949377
Iteration 9, loss = 0.12925744
Iteration 10, loss = 0.12849453
Iteration 11, loss = 0.12815726
Iteration 12, loss = 0.12770019
Iteration 13, loss = 0.12742685
Iteration 14, loss = 0.12721522
Iteration 15, loss = 0.12703030
Iteration 16, loss = 0.12680971
Iteration 17, loss = 0.12661886
Iteration 18, loss = 0.12656824
Iteration 19, loss = 0.12632815
Iteration 20, loss = 0.12635004
Iteration 21, loss = 0.12603595
Iteration 22, loss = 0.12608076
Iteration 23, loss = 0.12600323
Iteration 24, loss = 0.12584241
Iteration 25, loss = 0.12596304
Iteration 26, loss = 0.12570918
Iteration 27, loss = 0.12561201
Iteration 28, loss = 0.12565901
Iteration 29, loss = 0.12574677
Training loss did not improve more than tol=0.000100 for two consecutive epochs. 

ticket_id
284932    0.075411
285362    0.065755
285361    0.057434
285338    0.073871
285346    0.056371
285345    0.073819
285347    0.049144
285342    0.251450
285530    0.062923
284989    0.023007
285344    0.048284
285343    0.065426
285340    0.064884
285341    0.047878
285349    0.050194
285348    0.066980
284991    0.021322
285532    0.024668
285406    0.024637
285001    0.029026
285006    0.087674
285405    0.059747
285337    0.023704
285496    0.047139
285497    0.071540
285378    0.059675
285589    0.020201
285585    0.071769
285501    0.055071
285581    0.062898
            ...   
376367    0.005303
376366    0.017190
376362    0.083545
376363    0.610680
376365    0.005052
376364    0.016385
376228    0.017813
376265    0.020448
376286    0.730506
376320    0.021392
376314    0.023115
376327    0.910196
376385    0.845227
376435    0.417402
376370    0.497180
376434    0.087066
376459    0.133916
376478    0.000499
376473    0.035877
376484    0.002616
376482    0.020181
37